### **Concepto General**

Frame Differencing pertence al tipo de técnicas conocidas como "Temporal/Frame Difference" (Diferencia Temporal o Diferencia de Cuadros). Este tipo de tecnica se basan en la comparacion de cuadros consecutivos en una secuencia de video para detectar cambios que indicar movimiento.

### **Concepto**

La diferencia temporal se refiere a la ténica que utiliza la comparación de cuadros en diferentes momentos para detectar cambios en una secuencia de video. El principio basico es que los cambios en la escena entre dos cuadros consecutivos indican movimiento

### **Pasos clave**

1.- Captura de cuadros: Se capturan cuadros consecutivos de la secuencia de video.

2.- Cálculo de la diferencia: Se calcula la diferencia en los valores de los pixeles entres los cuadros consecutivos 

3.- Umbralizacion: Se aplica un umbral para distinguir entre cambios significativos (movimiento) y ruido

4.- Generación de mascara de movimiento: Se genera una mascara binaria que indica las areas de movimiento

### **Ventajas**
<ul>
    <li>Simplificidad
    <li>Rapidez 
    <li>Eficiencia en Escenas Estáticas
</ul>

### **Desventajas**
<ul>
    <li>Sensibilidad al ruido
    <li>Problemas con iluminación 
    <li>Occlusión y sombras
</ul>

### **Ejemplos de uso**
<ul>
    <li>Vigilancia de seguridad
    <li>Deteccion de Tráfico
    <li>Aplicaciones medica
</ul>



Empezaremos utilizando el modelo de reconomiento de movimiento conocido como "Frame Differencing" de la siguiente manera

In [3]:
# Librerias necesarias
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
import tensorflow_hub as hub

2024-07-31 21:02:24.796190: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 21:02:24.823539: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 21:02:24.833691: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-31 21:02:24.852527: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-31 21:02:26.130878: W tensorflow/compiler/tf2

**Para guardar el video usando un modelo de diferencias**

In [12]:
def get_significant_bits(image, num_bits=4):
    shift_amount = 8 - num_bits
    return (image >> shift_amount) << shift_amount

def get_least_significant_bits(image, num_bits=4):
    mask = (1 << num_bits) - 1
    return image & mask

#Abre el video 
ruta = '../../Persona8/persona8_9_f.webm'
cap = cv2.VideoCapture(ruta)
#Lee el primer cuadro del video 
ret, frame = cap.read()
#convertimos la imagen a escala de grises
if not ret:
    print("No se encontro video") 
image1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#Obtenemos las dimensiones del cuadro en escala de grises
row, col = image1.shape

#creamos una imagen en blanco con las mismas dimensiones 
res = np.zeros([row, col, 1], np.uint8)

#Intensidades para la mascara binaria 

#obtencion de los cuadros por segundo del video 
fps = cap.get(cv2.CAP_PROP_FPS)

#Configuracion para la escritura del video de saldia
out  = cv2.VideoWriter("personaFrameDifferencing4less.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (col, row), isColor=False)

image1 =  get_least_significant_bits(image1) * (255 // ((1 << 4) - 1))

while True:
    _, image2 = cap.read()
    if image2 is None:
        break

    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    image2 = get_least_significant_bits(image2) * (255 // ((1 << 4) - 1))
    res = cv2.absdiff(image1, image2)
    
    cv2.imshow('image',res)
    out.write(res)
    image1 = image2
    if cv2.waitKey(1) & 0xFF == 27:
        break

out.release()
cap.release()
cv2.destroyAllWindows()

Para usar el modelo y viendo el video con un bucle sin fin

In [49]:
#Abre el video  
ruta = "../../Persona8/persona8_9_f.webm"
cap = cv2.VideoCapture(ruta)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frames = []
while cap.isOpened():
    ret, fram = cap.read()
    if not ret:
        break
    frames.append(frame)

mediana = np.median(frames, axis=0).astype(np.uint8)

cap = cv2.VideoCapture(ruta)
#Lee el primer cuadro del video 
_, frame = cap.read()
#convertimos la imagen a escala de grises
image1 = mediana.astype(np.uint8)
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
print(image1.shape)
#Obtenemos las dimensiones del cuadro en escala de grises
row, col = image1.shape

#obtencion de los cuadros por segundo del video 
fps = cap.get(cv2.CAP_PROP_FPS)

#Configuracion para la escritura del video de saldia
out  = cv2.VideoWriter("personaFrameDifferencingMediana.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (col, row), isColor=False)
ancho_nuevo = 1600
alto_nuevo = 900
while True:
    _, image2 = cap.read()
    if image2 is None:
        #cap = cv2.VideoCapture(ruta)
        #Lee el primer cuadro del video 
        #_, image2 = cap.read()
        break
    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    # Normalizar las imágenes

    # Calcular la diferencia
    res = cv2.absdiff(image2, image1)

    # Escalar la diferencia de vuelta al rango de 0 a 255
    out.write(res)
    res = cv2.resize(res, (ancho_nuevo, alto_nuevo) )
    cv2.imshow('image',res)

    if cv2.waitKey(1) & 0xFF == 27:
        break

out.release()
cap.release()
cv2.destroyAllWindows()

(1080, 1920)


### **Usando mediapipe**

In [3]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils
ruta = "../../Persona8/persona8_9_f.webm"
cap = cv2.VideoCapture(ruta)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow('Pose Estimation', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1722323425.764294   28133 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1722323425.765852   28216 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 22.3.6), renderer: Mesa Intel(R) HD Graphics 530 (SKL GT2)
W0000 00:00:1722323425.881641   28207 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1722323425.900571   28212 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


### **Encontrando contorno con respecto a colores**

In [16]:
ruta = "../../Persona8/persona8_9_f.webm"
cap = cv2.VideoCapture(ruta)



while cap.isOpened():
    ret, image = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplicar un umbral para obtener una imagen binaria
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

    # Encontrar contornos
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear una máscara vacía
    mask = np.zeros_like(gray)

    # Dibujar los contornos en la máscara
    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)

    # Aplicar la máscara a la imagen original
    result = cv2.bitwise_and(image, image, mask=mask)

    # Mostrar la imagen resultante
    cv2.imshow('Silueta', result)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break


cv2.destroyAllWindows()

### **

In [17]:

# Configuración de MediaPipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Inicializar el modelo de pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Cargar el video
video_path = "../../Persona8/persona8_9_f.webm"
cap = cv2.VideoCapture(video_path)

while True:
    # Leer el siguiente cuadro del video
    ret, frame = cap.read()

    # Si no se puede leer más cuadros, salir del bucle
    if not ret:
        break

    # Convertir la imagen de BGR a RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Realizar la detección de pose
    results = pose.process(rgb_frame)

    # Crear una máscara negra del mismo tamaño que el cuadro
    mask = np.zeros_like(frame)

    # Dibujar las marcas y conexiones de la pose
    if results.pose_landmarks:
        # Obtener puntos clave y convertirlos en coordenadas
        h, w, _ = frame.shape
        points = []
        for landmark in results.pose_landmarks.landmark:
            x, y = int(landmark.x * w), int(landmark.y * h)
            points.append((x, y))
        
        # Definir las conexiones entre los puntos (de la pose)
        connections = [
            (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
            (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),
            (mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST),
            (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW),
            (mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
            (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP),
            (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP),
            (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP),
            (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE),
            (mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE),
            (mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE),
            (mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE),
        ]

        for start, end in connections:
            start_point = points[start.value]
            end_point = points[end.value]
            cv2.line(mask, start_point, end_point, (255, 255, 255), 5)
        
        # Crear una máscara de relleno usando un relleno convexo
        points_array = np.array(points, np.int32)
        if len(points_array) > 0:
            cv2.fillConvexPoly(mask, points_array, (255, 255, 255))

    # Aplicar la máscara al cuadro original
    result = cv2.bitwise_and(frame, mask)

    # Mostrar el resultado
    cv2.imshow('Silueta', result)

    # Salir del bucle si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar los recursos
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1722478747.604356   54740 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1722478747.608639   56168 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 22.3.6), renderer: Mesa Intel(R) HD Graphics 530 (SKL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1722478747.709759   56153 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1722478747.739855   56158 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/home/kinlo/.local/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please

In [18]:
# Python program to implement  
# Webcam Motion Detector 
  
# importing OpenCV, time and Pandas library 
import cv2, time, pandas 
# importing datetime class from datetime library 
from datetime import datetime 
  
# Assigning our static_back to None 
static_back = None
  
# List when any moving object appear 
motion_list = [ None, None ] 

video_path = "../../Persona8/persona8_9_f.webm"
# Time of movement 
time = [] 
  
# Initializing DataFrame, one column is start  
# time and other column is end time 
df = pandas.DataFrame(columns = ["Start", "End"]) 
  
# Capturing video 
video = cv2.VideoCapture(video_path) 
  
# Infinite while loop to treat stack of image as video 
while True: 
    # Reading frame(image) from video 
    check, frame = video.read() 
    if not check:
        break
  
    # Initializing motion = 0(no motion) 
    motion = 0
  
    # Converting color image to gray_scale image 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
  
    # Converting gray scale image to GaussianBlur  
    # so that change can be find easily 
    gray = cv2.GaussianBlur(gray, (21, 21), 0) 
  
    # In first iteration we assign the value  
    # of static_back to our first frame 
    if static_back is None: 
        static_back = gray 
        continue
  
    # Difference between static background  
    # and current frame(which is GaussianBlur) 
    diff_frame = cv2.absdiff(static_back, gray) 
  
    # If change in between static background and 
    # current frame is greater than 30 it will show white color(255) 
    thresh_frame = cv2.threshold(diff_frame, 30, 255, cv2.THRESH_BINARY)[1] 
    thresh_frame = cv2.dilate(thresh_frame, None, iterations = 2) 
  
    # Finding contour of moving object 
    cnts,_ = cv2.findContours(thresh_frame.copy(),  
                       cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
  
    for contour in cnts: 
        if cv2.contourArea(contour) < 10000: 
            continue
        motion = 1
  
        (x, y, w, h) = cv2.boundingRect(contour) 
        # making green rectangle around the moving object 
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3) 
  
    # Appending status of motion 
    motion_list.append(motion) 
  
    motion_list = motion_list[-2:] 
  
    # Appending Start time of motion 
    if motion_list[-1] == 1 and motion_list[-2] == 0: 
        time.append(datetime.now()) 
  
    # Appending End time of motion 
    if motion_list[-1] == 0 and motion_list[-2] == 1: 
        time.append(datetime.now()) 
  
    # Displaying image in gray_scale 
    #cv2.imshow("Gray Frame", gray) 
  
    # Displaying the difference in currentframe to 
    # the staticframe(very first_frame) 
    #cv2.imshow("Difference Frame", diff_frame) 
  
    # Displaying the black and white image in which if 
    # intensity difference greater than 30 it will appear white 
    cv2.imshow("Threshold Frame", thresh_frame) 
  
    # Displaying color frame with contour of motion of object 
    #cv2.imshow("Color Frame", frame) 
    # if q entered whole process will stop 
    # Salir del bucle si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
  
video.release() 
  
# Destroying all the windows 
cv2.destroyAllWindows() 

### **Binarizacion**

In [50]:
video_path = 'personaFrameDifferencingMediana.avi'
cap = cv2.VideoCapture(video_path)
umbral = 60
ret, frame = cap.read()

if not ret:
    print("hola")

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("personaFrameDifferencingBinari.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height), isColor=False)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if len(frame.shape) == 3:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, binary_frame = cv2.threshold(frame, umbral, 255, cv2.THRESH_BINARY)
    out.write(binary_frame)
    cv2.imshow('Video binarizado', binary_frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break 

out.release()
cap.release()
cv2.destroyAllWindows()

## **Morphological Transformations**

Primero empezando con **Opening** 

In [51]:
kernel = np.ones((5,5), np.uint8)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2,2))
video_path = "personaFrameDifferencingBinari.avi"
cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("personaFrameDifferencingOpeningEclipse_2.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height), isColor=False)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if len(frame.shape) == 3:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    opening = cv2.morphologyEx(frame, cv2.MORPH_OPEN, kernel=kernel)
    cv2.imshow('Video binarizado', opening)
    out.write(opening)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break 

out.release()
cap.release()
cv2.destroyAllWindows()


Ahora con el **Closing**

In [52]:
kernel = np.ones((100, 100), np.uint8)
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (50,50))
video_path = "personaFrameDifferencingOpening.avi"
cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("personaFrameDifferencingClosingCross50_50.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height), isColor=False)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if len(frame.shape) == 3:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    closing = cv2.morphologyEx(frame, cv2.MORPH_CLOSE, kernel=kernel)
    cv2.imshow('Video', closing)
    out.write(closing)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break 

out.release()
cap.release()
cv2.destroyAllWindows()

## **Edges con Canny**

In [53]:
#Abre el video 
ruta = "../../Persona8/persona8_9_f.webm"
cap = cv2.VideoCapture(ruta)
#Lee el primer cuadro del video 
_, frame = cap.read()
#convertimos la imagen a escala de grises
image1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#Obtenemos las dimensiones del cuadro en escala de grises
row, col = image1.shape

#creamos una imagen en blanco con las mismas dimensiones 
res = np.zeros([row, col, 1], np.uint8)

#Intensidades para la mascara binaria 

#obtencion de los cuadros por segundo del video 
fps = cap.get(cv2.CAP_PROP_FPS)

#Configuracion para la escritura del video de saldia
out  = cv2.VideoWriter("personaFrameEdges.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (col, row), isColor=False)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    edges = cv2.Canny(frame, 5, 50)

    cv2.imshow('Edges', edges)
    out.write(edges)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
out.release()
cap.release()
cv2.destroyAllWindows(  )

## **Usando Sobel**

In [18]:
#Abre el video 
ruta = "../../Persona8/persona8_9_f.webm"
cap = cv2.VideoCapture(ruta)
#Lee el primer cuadro del video 
_, frame = cap.read()
#convertimos la imagen a escala de grises
image1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#Obtenemos las dimensiones del cuadro en escala de grises
row, col = image1.shape

#creamos una imagen en blanco con las mismas dimensiones 
res = np.zeros([row, col, 1], np.uint8)

#Intensidades para la mascara binaria 

#obtencion de los cuadros por segundo del video 
fps = cap.get(cv2.CAP_PROP_FPS)

#Configuracion para la escritura del video de saldia
out  = cv2.VideoWriter("personaFrameEdges.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (col, row), isColor=False)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_blur = cv2.GaussianBlur(frame_gray, (5,5), 0)

    laplacian = cv2.Laplacian(frame_blur, cv2.CV_64F)
    cv2.imshow('Edge sobely', laplacian)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

out.release()
cap.release()
cv2.destroyAllWindows()

## **Contour Detection**

In [2]:

def img_to_polygons(image):
    cordnt_list = []

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to create a binary image
    _, th2 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(th2, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    # Extract coordinates of white regions
    white_regions_coordinates = []
    for contour in contours:
        coordinates = contour.squeeze().astype(float).tolist()
        white_regions_coordinates.append(coordinates)

    # Print the coordinates of each white region
    for i, region in enumerate(white_regions_coordinates):
        if type(region[0]) is list:
            if len(region) > 2:
                # Calculate the area of the contour
                area = cv2.contourArea(np.around(np.array([[pnt] for pnt in region])).astype(np.int32))
                if area > 100:
                    # Convert coordinates to the required format
                    crdnts = [{'x': i[0], 'y': i[1]} for i in region]
                    cordnt_list.append(crdnts)

    return cordnt_list

#Abre el video 
ruta = "../../Persona8/persona8_9_f.webm"
cap = cv2.VideoCapture(ruta)
#Lee el primer cuadro del video 
ret, frame = cap.read()
#convertimos la imagen a escala de grises
if ret:
    image1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #Obtenemos las dimensiones del cuadro en escala de grises
    row, col = image1.shape

    #creamos una imagen en blanco con las mismas dimensiones 
    res = np.zeros([row, col, 1], np.uint8)

    #Intensidades para la mascara binaria 

    #obtencion de los cuadros por segundo del video 
    fps = cap.get(cv2.CAP_PROP_FPS)
    out  = cv2.VideoWriter("personaFrameEdgesContor.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (col, row), isColor=False)

else:
    print("Erorr")


# Process each file in the input directory
while cap.isOpened():
    # Input and output file paths

    ret, img = cap.read()
    if not ret:
        break
    # Extract polygons from the image
    polygons = img_to_polygons(img)

    # Get the height and width of the image
    height, width = img.shape[:2]

    # Create a black image with the same dimensions as the input
    black_image = np.zeros((height, width, 3), dtype=np.uint8)

    # Draw polygons on the black image
    for ply in polygons:
        ply_list = [[pnt['x'], pnt['y']] for pnt in ply]

        # Define the vertices of the polygon
        vertices = np.array(ply_list, dtype=np.int32)

        # Draw the polygon on the black image
        cv2.polylines(black_image, [vertices], isClosed=True, color=(0, 255, 0), thickness=2)

    cv2.imshow('Poligonos', black_image)
    out.write(black_image)
    # Salir del bucle con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
out.release()
# Liberar los recursos
cap.release()
cv2.destroyAllWindows()


In [55]:
videoEdge = "personaFrameEdges.avi"
videoClose = "personaFrameDifferencingClosingCross50_50.avi"

capEdge = cv2.VideoCapture(videoEdge)
capClose = cv2.VideoCapture(videoClose)

width = int(capEdge.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capEdge.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps= int(capEdge.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('FinalEdge.mp4', fourcc, fps, (width, height))

while capEdge.isOpened():
    ret1, frame1 = capEdge.read()
    ret2, frame2 = capClose.read()

    if not ret1 or not ret2:
        break

    frame1 = cv2.resize(frame1, (width, height))
    frame2 = cv2.resize(frame2, (width, height))

    bitwise_and = cv2.bitwise_and(frame1, frame2)
    out.write(bitwise_and)
    cv2.imshow("Bit_and", bitwise_and)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

out.release()
capEdge.release()
capClose.release()
cv2.destroyAllWindows()



In [11]:
def get_significant_bits(image, num_bits=4):
    # Reduce to the most significant bits
    shift_amount = 8 - num_bits
    return (image >> shift_amount) << shift_amount

def get_least_significant_bits(image, num_bits=4):
    # Crear una máscara para los bits menos significativos
    mask = (1 << num_bits) - 1
    return image & mask

#Abre el video 
ruta = '../../Persona8/persona8_9_f.webm'
cap = cv2.VideoCapture(ruta)
#Lee el primer cuadro del video 
ret, frame = cap.read()
#convertimos la imagen a escala de grises
if not ret:
    print("No se encontro video") 
image1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#Obtenemos las dimensiones del cuadro en escala de grises
row, col = image1.shape

#creamos una imagen en blanco con las mismas dimensiones 
res = np.zeros([row, col, 1], np.uint8)

#Intensidades para la mascara binaria 

#obtencion de los cuadros por segundo del video 
fps = cap.get(cv2.CAP_PROP_FPS)

#Configuracion para la escritura del video de saldia
#out  = cv2.VideoWriter("personaFrameDifferencing4less.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (col, row), isColor=False)


image1 =  get_least_significant_bits(image1) * (255 // ((1 << 4) - 1))


cv2.imshow("imagen", image1)
cv2.imwrite("4LSB.png",image1)
cv2.waitKey(0)

out.release()
cap.release()
cv2.destroyAllWindows()